In [ ]:
import numpy as np
import multiprocessing
from tensorflow import keras



In [ ]:
# Define the custom generator that uses multiprocessing to preprocess the data in parallel
class ParallelGenerator(keras.utils.Sequence):
    
    def __init__(self, x, y, batch_size=32, num_workers=4):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pool = multiprocessing.Pool(processes=num_workers)
        self.indexes = np.arange(len(self.x))
        np.random.shuffle(self.indexes)
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_x = self.x[batch_indexes]
        batch_y = self.y[batch_indexes]
        results = self.pool.map_async(preprocess_image, batch_x)
        batch_x = np.array(results.get())
        return batch_x, batch_y
    
    def on_epoch_end(self):
        np.random.shuffle(self.indexes)
        
    def __del__(self):
        self.pool.close()
        self.pool.join()




In [ ]:
# Define the function to preprocess an image using a separate process
def preprocess_image(img):
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=-1)
    return img



In [ ]:
# Load the Fashion-MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()



In [ ]:
# Preprocess the dataset
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)




In [ ]:
# Define the CNN model
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model using the parallel generator
train_generator = ParallelGenerator(x_train, y_train, batch_size=32, num_workers=4)
model.fit(train_generator, epochs=5)



Epoch 1/5
1875/1875 [==============================] - 75s 39ms/step - loss: 0.9285 - accuracy: 0.6569
Epoch 2/5
1875/1875 [==============================] - 71s 38ms/step - loss: 0.6464 - accuracy: 0.7582
Epoch 3/5
1875/1875 [==============================] - 71s 38ms/step - loss: 0.5781 - accuracy: 0.7826
Epoch 4/5
1875/1875 [==============================] - 71s 38ms/step - loss: 0.5298 - accuracy: 0.8009
Epoch 5/5
1875/1875 [==============================] - 71s 38ms/step - loss: 0.4971 - accuracy: 0.8160


In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 3s 9ms/step - loss: 279.0136 - accuracy: 0.3412
Test accuracy: 0.34119999408721924
